In [201]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [202]:
def calculate_percentage_increase(initial_value, final_value):
    percentage_increase = ((final_value - initial_value) / initial_value) * 100
    return "{:.1f}%".format(percentage_increase)

In [203]:
clay_womens=pd.read_sql_query(
    f"Select DISTINCT * From AllMatches where Sex='Womens' and surface = 'Clay'",con=devengine,)
clay_womens[['Player_1_Odds','Player_2_Odds']]=clay_womens[['Player_1_Odds','Player_2_Odds']].astype(float)
clay_womens['Fav_Odds']=clay_womens[['Player_1_Odds','Player_2_Odds']].min(axis=1)
clay_womens['Dog_Odds']=clay_womens[['Player_1_Odds','Player_2_Odds']].max(axis=1)
clay_womens['max_rank']=clay_womens[['Player_1_Rank','Player_2_Rank']].max(axis=1)
clay_womens['min_rank']=clay_womens[['Player_1_Rank','Player_2_Rank']].min(axis=1)
clay_womens['Winner']=clay_womens.apply(lambda x: 'Dog' if x['Player_1_Odds']>x['Player_2_Odds'] else 'Fav', axis=1)

In [204]:
bankroll=1000
STAKE=100
clay_womens=clay_womens[(clay_womens['Date'])>'2022-01-01']
#clay_womens=clay_womens[((clay_womens['min_rank'])<150)]
#clay_womens=clay_womens[((clay_womens['max_rank'])<150)]
#clay_womens=clay_womens[(clay_womens['Dog_Odds']).ge(2.1)&(clay_womens['Dog_Odds']).le(2.2)]
clay_womens['Win']=clay_womens.apply(lambda x: True if x['Winner']=='Dog' else False, axis=1)
clay_womens['Profit']=clay_womens.apply(lambda x: -STAKE if x['Win']==False else (x['Dog_Odds']*STAKE)-STAKE,axis=1)
roi=calculate_percentage_increase(bankroll,bankroll+clay_womens['Profit'].sum())
print(clay_womens['Profit'].sum(),roi,len(clay_womens))


-27354.0 -2735.4% 1736


In [205]:
for x in range(1,4):
    for y in range(0,10):
        lower=x+(y*0.1)
        upper=x+((y+1)*0.1)
        #print(lower,upper)
        test1=clay_womens[(clay_womens['Dog_Odds'].ge(lower))&(clay_womens['Dog_Odds'].le(upper))].copy()
        games=len(test1)
        if games > 0:
            percent=len(test1[test1['Winner']!='Fav'])/len(test1)
            STAKE=100
            test1['Profit']=test1.apply(lambda x: -STAKE if x['Winner']=='Fav' else (STAKE * x['Dog_Odds'])-STAKE,axis=1)
            if test1['Profit'].sum() >0 and len(test1) >0:
                print(lower,upper,percent,games)
                print(test1['Profit'].sum())

2.0 2.1 0.5 142
338.99999999999966
2.6 2.7 0.38202247191011235 89
130.0
3.1 3.2 0.375 56
1010.0
3.2 3.3 0.34146341463414637 41
449.0
3.4 3.5 0.35294117647058826 34
738.0
3.6 3.7 0.2777777777777778 36
34.0
3.8 3.9 0.28 25
183.0
